# Importing libraries 

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os
import cvzone
from PIL import Image
from sort import *

# Model 

In [2]:
model = YOLO('new_model.pt')

## Classnames 

In [3]:
classnames = ['Columnaris Disease', 'EUS Disease', 'Gill Disease', 'Healthy Fish', 'streptococcus Disease']

In [6]:
classnames = ['disease', 'good', 'goods']

# Sample inference 

In [7]:
image = cv2.imread('diseased fish 2.jpg')
result = model(image, show=False)[0]
for r in result.boxes.data.tolist():
    x1,y1,x2,y2,score,class_id = r
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

plt.imshow(image)
plt.title(classnames[int(class_id)])
plt.show()


0: 544x640 1 disease, 162.9ms
Speed: 10.0ms preprocess, 162.9ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)


In [5]:
result.names.values()

dict_values(['disease', 'good'])

In [8]:
image = cv2.imread('diseased fish 1.jpeg')
result = model(image, show=False)[0]
for r in result.boxes.data.tolist():
    x1,y1,x2,y2,score,class_id = r
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

plt.imshow(image)
plt.title(classnames[int(class_id)])
plt.show()


0: 256x640 1 disease, 2 goods, 136.4ms
Speed: 5.3ms preprocess, 136.4ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 640)


In [9]:
image = cv2.imread('good fish.jpeg')
result = model(image, show=False)[0]
for r in result.boxes.data.tolist():
    x1,y1,x2,y2,score,class_id = r
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

plt.imshow(image)
plt.title(classnames[int(class_id)])
plt.show()


0: 448x640 4 goods, 127.1ms
Speed: 5.0ms preprocess, 127.1ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)


In [5]:
result.names.values()

dict_values(['Columnaris Disease', 'EUS Disease', 'Gill Disease', 'Healthy Fish', 'streptococcus Disease'])

# Video 

In [18]:
# Tracking
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

limits = [400, 297, 673, 297]
totalCount = []

cap = cv2.VideoCapture('good fish video.mp4')

while True:
    
    ret, frame = cap.read()
    detections = np.empty((0, 5))
    results = model(frame, show=False)[0]
    
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255),4)
        
        cv2.putText(frame, classnames[int(class_id)], (40,50), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 1)
        
        if score > 0.7:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 5)
            cv2.putText(frame, f'{score:.2f}', (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2,
                                cv2.LINE_AA)
            currentArray = np.array([x1, y1, x2, y2, score])
            detections = np.vstack((detections, currentArray))

    resultsTracker = tracker.update(detections)
    
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(frame, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(frame, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)
            
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

    
        if totalCount.count(id) == 0:
            totalCount.append(id)

    cv2.putText(frame,str(len(totalCount)),(10,10),cv2.FONT_HERSHEY_PLAIN,5,(50,50,255),8)
    cv2.imshow('Output', frame)
        
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


0: 320x640 (no detections), 106.0ms
Speed: 7.0ms preprocess, 106.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 90.0ms
Speed: 5.5ms preprocess, 90.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 79.1ms
Speed: 4.0ms preprocess, 79.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 73.2ms
Speed: 3.0ms preprocess, 73.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 73.9ms
Speed: 3.0ms preprocess, 73.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 75.4ms
Speed: 3.0ms preprocess, 75.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 76.0ms
Speed: 4.0ms preprocess, 76.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 77.4ms
Speed: 3.0ms preprocess, 77.4ms

In [20]:
cap.release()
cv2.destroyAllWindows()

In [18]:
import cv2
import numpy as np
import cvzone

tracker = cv2.legacy.TrackerCSRT_create()  # Creating a CSRT tracker
# Initialize an empty list to store the IDs of tracked objects
totalCount = []

cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    detections = []
    results = model(frame, show=False)[0]
    
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255),4)
        
        cv2.putText(frame, classnames[int(class_id)], (40,50), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 1)
        
        if score > 0.7:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 5)
            cv2.putText(frame, f'{score:.2f}', (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2,
                        cv2.LINE_AA)
            # Append the detected bounding box coordinates and score to the detections list
            detections.append((x1, y1, x2 - x1, y2 - y1))

    # Convert detections to numpy array
    detections = np.array(detections)
    
    # Update the tracker with the detections
    for detection in detections:
        success, box = tracker.update(frame, detection)
        if success:
            x, y, w, h = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 255), 3)
    
    # Display the total count of tracked objects
    cv2.putText(frame,str(len(totalCount)),(255,100),cv2.FONT_HERSHEY_PLAIN,5,(50,50,255),8)
    cv2.imshow('Output', frame)
        
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


AttributeError: module 'cv2' has no attribute 'legacy'

In [19]:
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)
totalCount = []

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    detections = []
    results = model(frame, show=False)[0]
    
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 4)
        cv2.putText(frame, classnames[int(class_id)], (40, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 1)
        
        if score > 0.7:
            detections.append([x1, y1, x2, y2, score])

    detections = np.array(detections)
    trackers = tracker.update(detections)
    
    for trk in trackers:
        x1, y1, x2, y2, id = trk.astype(np.int32)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 5)
        cv2.putText(frame, str(int(id)), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
        
        if id not in totalCount:
            totalCount.append(id)

    cv2.putText(frame, str(len(totalCount)), (255, 100), cv2.FONT_HERSHEY_PLAIN, 5, (50, 50, 255), 8)
    cv2.imshow('Output', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 85.0ms
Speed: 3.0ms preprocess, 85.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 87.1ms
Speed: 2.0ms preprocess, 87.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 75.1ms
Speed: 2.0ms preprocess, 75.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 71.5ms
Speed: 2.0ms preprocess, 71.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 75.1ms
Speed: 2.2ms preprocess, 75.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 69.2ms
Speed: 1.0ms preprocess, 69.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 EUS Disease, 65.0ms
Speed: 1.0ms preprocess, 65.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 EUS Disease, 66.1ms
Speed: 1.0ms preprocess, 66.1ms infer

IndexError: index 1 is out of bounds for axis 1 with size 1